<h5>Libraries</h5>

In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


<h5>Defining paths and labeling classes</h5>

In [2]:
dataset_path = 'processed_images'

class_labels = {
    'Cyst': 0,
    'Normal': 1,
    'Stone': 2,
    'Tumor': 3
}

<h5>Preprocessing of CT images and removal of black regions</h5>

In [3]:
def crop_black_borders(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(thresh)
    x, y, w, h = cv2.boundingRect(coords)
    cropped = img[y:y+h, x:x+w]
    return cropped

<h5>Extracting features from images</h5>

In [4]:
def extract_features(image, resize_dim=(200, 200), patch_size=10):
    cropped = crop_black_borders(image)
    resized = cv2.resize(cropped, resize_dim)
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient = cv2.magnitude(gx, gy).astype(np.uint8)

    h, w = gray.shape
    ph, pw = h // patch_size, w // patch_size

    feature_vector = []

    for i in range(0, h, ph):
        for j in range(0, w, pw):
            patch_gray = gray[i:i+ph, j:j+pw]
            patch_grad = gradient[i:i+ph, j:j+pw]

            hist_gray = cv2.calcHist([patch_gray], [0], None, [8], [0, 256]).flatten()
            hist_grad = cv2.calcHist([patch_grad], [0], None, [8], [0, 256]).flatten()

            features = np.concatenate((hist_gray, hist_grad))
            feature_vector.extend(features)

    return np.array(feature_vector)


<h5>Data reading and feature set construction</h5>

In [11]:
X = []
y = []

for class_name, label in class_labels.items():
    class_dir = os.path.join(dataset_path, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                features = extract_features(img)
                X.append(features)
                y.append(label)

print(X, y)


[array([209.,  26.,  23., ...,  11.,  12.,  11.], dtype=float32), array([390.,   5.,   3., ...,   5.,   3.,   2.], dtype=float32), array([400.,   0.,   0., ...,   1.,   0.,   2.], dtype=float32), array([389.,   6.,   5., ...,   1.,   0.,   2.], dtype=float32), array([385.,  10.,   5., ...,   2.,   1.,   1.], dtype=float32), array([386.,   8.,   6., ...,   0.,   1.,   0.], dtype=float32), array([384.,  11.,   5., ...,   0.,   0.,   1.], dtype=float32), array([400.,   0.,   0., ...,  13.,   9.,  11.], dtype=float32), array([400.,   0.,   0., ...,  16.,   8.,  10.], dtype=float32), array([382.,  12.,   6., ...,   0.,   0.,   1.], dtype=float32), array([379.,  13.,   8., ...,   0.,   1.,   0.], dtype=float32), array([378.,  12.,  10., ...,   2.,   0.,   2.], dtype=float32), array([377.,  11.,  11., ...,   0.,   1.,   1.], dtype=float32), array([400.,   0.,   0., ...,   1.,   0.,   0.], dtype=float32), array([376.,  12.,  12., ...,   1.,   0.,   1.], dtype=float32), array([378.,  12.,  10.,

In [12]:
from tqdm import tqdm
import os
import cv2
import numpy as np

X = []
y = []

# Estimate total number of images (optional but helps tqdm show progress better)
total_images = sum(
    len([f for f in os.listdir(os.path.join(dataset_path, class_name)) if f.endswith('.jpg')])
    for class_name in class_labels
)

with tqdm(total=total_images, desc="Processing images") as pbar:
    for class_name, label in class_labels.items():
        class_dir = os.path.join(dataset_path, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg'):
                img_path = os.path.join(class_dir, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    features = extract_features(img)
                    X.append(features)
                    y.append(label)
                pbar.update(1)  # update progress bar

X = np.array(X)
y = np.array(y)


Processing images: 100%|██████████| 12446/12446 [03:29<00:00, 59.33it/s] 


<h5>Normalizing features and transforming labels</h5>

In [13]:
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # برای LSTM

y_categorical = to_categorical(y, num_classes=4)


<h5>Splitting the data into training and validation sets</h5>

In [14]:
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_categorical,
    test_size=0.2,
    random_state=42,
    stratify=y
)


<h5>Construction of an LSTM model</h5>
<p>can be different</p>

In [15]:
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    LSTM(200),
    Dropout(0.5),
    Dense(4, activation='softmax')
])


<h5>Compile and Training Model</h5>
<p>Configurable for experimenting with different architectures<p>

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
 64/312 ━━━━━━━━━━━━━━━━━━━━ 9:39 2s/step - accuracy: 0.3496 - loss: 1.3316

KeyboardInterrupt: 

<h5>Visualization of accuracy and loss metrics</h5>
<p>The number of epochs, batch size, and patience are configurable<p>

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

plt.tight_layout()
plt.show()
